## Faqs Bot

This bot will convert FAQ's to bots

In [2]:
from bs4 import BeautifulSoup
import requests

page = requests.get("https://www.cdc.gov/coronavirus/2019-ncov/faq.html")
soup = BeautifulSoup(page.content, 'html.parser')

In [3]:
intentExamplesScraped = {}
botMessagesScraped = {"basic":{},"test":{}}

testCase = {"faqListed":{
  "conversation": [
  ],
  "description": "Contains all Faq's and expected answers on the cdc website"
}}


def removeSpecialCharactersWithSpaces(text):
    return ''.join(e for e in text if e.isalnum())
    

for el in soup.find_all(class_="card card-accordion"):
    
    ques = el.find(class_="card-title btn btn-link")    
    ans  = el.find(class_="card-body")
    
    print("Question : " , ques.text)
    print("Answer : ",  ans.text)
    
    
    intentName = removeSpecialCharactersWithSpaces(ques.text)
    intentExamplesScraped[intentName] = [ques.text.replace("?"," ?")]*10
    botMessagesScraped["basic"][intentName] = {"basic" :[]}
    botMessagesScraped["basic"][intentName]["basic"].append(ans.text)  
    
    botMessagesScraped["basic"][intentName] = {"basic" :[]}
    botMessagesScraped["basic"][intentName]["basic"].append(ans.text)
    
    testCase["faqListed"]["conversation"].append([ques.text.replace("?"," ?"),ans.text])
    
    print("********\n")

Question :  What is COVID-19?
Answer :  COVID-19 is a disease caused by a virus called SARS-CoV-2. Most people with COVID-19 have mild symptoms, but some people can become severely ill. Although most people with COVID-19 get better within weeks of illness, some people experience post-COVID conditions. Post-COVID conditions are a wide range of new, returning, or ongoing health problems people can experience more than four weeks after first being infected with the virus that causes COVID-19. Older people and those who have certain underlying medical conditions are more likely to get severely ill from COVID-19. Vaccines against COVID-19 are safe and effective.
********

Question :  How does the virus spread?
Answer :  COVID-19 spreads when an infected person breathes out droplets and very small particles that contain the virus. These droplets and particles can be breathed in by other people or land on their eyes, noses, or mouth. In some circumstances, they may contaminate surfaces they t

In [4]:
from simbots.Bot import Bot
from simbots.utils.builtInIntents import IntentSamples
from simbots.utils.builtInEntities import EntitySamples
import json
import yake


intentExamples= {
    
   'Greetings': IntentSamples.greetingSamples(),
    
    'BotName': IntentSamples.botNameSamples(),
    
    'Relatives': IntentSamples.relativeSamples(),
    
    'Age': IntentSamples.ageSamples(),
    
    'BirthPlace': IntentSamples.birthPlaceSamples(),
    
    'Abilities': IntentSamples.abilitiesSamples(), 
    
    "Bye":IntentSamples.byeSamples(),
    
    'Irrelevant': ['the weather is fine today','the sun rises in the east','the quick brown fox jumps over the red carpet',
               'the sun rises in the east',
               'What is love , baby dont hurt me ',
               'this is a new dawn a new day'],
    
}



botMessages = {
  "basic": {
      'Age': {
			'basic': ['I am two years old ', 'I am two']
		},
		'BotName': {
			'basic': ['I am CovidInfoBot', 'You can call me CovidInfoBot']
		},
		'Abilities': {
			'basic': ['I can tell you about Covid 19 or corona virus disease .']
		},
		'BirthPlace': {
			'basic': ['I am from India , India', 'I am from India', 'I am punjabi and i love food !']
		},
        "Greetings":{
              "basic" : ["Hi ","Hello "]  
        },
        "Bye":{
           "basic":["It was nice talking to you ! Take care, keep safe and wear a mask ."]  
        },
        "Relatives":{
            "basic":["No , I dont have any relatives ."]
            
        },
       'Irrelevant': {
			'basic': ['Im sorry Im not getting you :( ',
				'Im sorry could you please rephrase ?'
			]
		},
      
  }
    
}


entityExamples = {
    'GreetingsHelper': EntitySamples.greetingsHelper(),
    'ByeHelper': EntitySamples.byeHelper(),
    'Covid19':{
        'covid19':[{"type" : "regex","tag" :'case-insensitive',"pattern":"(corona virus disease)|(corona virus)|(covid-19)|(covid 19)|(covid19)|(coronavirus disease)|(covid disease)|(covid)"}]
    },
    'SarsCov2':{
        'sarscov2':[
            {
                "type":"regex",
                "tag":'case-insensitive',
                "pattern":"(SARS-CoV-2)|(SarsCov2)(Sars)"
                
            }
            
        ]
    }
    
}


def createExampleSentencesThroughYake(text,max_ngram_size = 4,minLength=0,probThreshold=0.85):
    language = "en"
    deduplication_threshold = 0.8
    numOfKeywords = 50
    keyWordExtractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
    returnedKeywords =[ ]
    text = ".".join([removeSpecialCharactersWithOutSpaces(el) for el in text.split(".")])
    keywords = keyWordExtractor.extract_keywords(text)
    
    for keyWord in keywords:
        if (len(keyWord[0].split() ) >=minLength) and keyWord[1]>probThreshold:
            returnedKeywords.append(keyWord[0])
    return returnedKeywords

def removeSpecialCharactersWithOutSpaces(text):
    return ''.join(e  if e.isalnum() or e==" " else " "  for e in text )
            
for intent in intentExamplesScraped.keys():
    
    # extract keywords from answers
    intentExamplesScraped[intent].extend(createExampleSentencesThroughYake(botMessagesScraped["basic"][intent]["basic"][0] ,probThreshold=0.3,minLength=2))
    # extract keywords from questions themselves
    
    intentExamplesScraped[intent].extend(createExampleSentencesThroughYake(intentExamplesScraped[intent][0] ,probThreshold=0))

for intent in intentExamplesScraped.keys():
    intentExamples[intent] = intentExamplesScraped[intent]
    
for message in botMessagesScraped["basic"].keys():
    botMessages["basic"][message] = botMessagesScraped["basic"][message]

In [5]:
from simbots.Bot import Bot
from simbots.utils.builtInIntents import IntentSamples
from simbots.utils.builtInEntities import EntitySamples
import json


newB = Bot(intentExamples, entityExamples, botMessages,testCases=testCase,
              confidenceLimit=0)

outputTheme = 'basic'

newB.run(theme = outputTheme)

Type in @@ to exit bot
@i to get intents
@c to get context 
@e to get entities
@t to add a test case
@etc to evaluate a test case
@eatc to evaluate all test cases
@ti to add intent
<1> User : hi
<1> basicBot : Hi 
<2> User : who might you be ?
<2> basicBot : You can call me CovidInfoBot
<3> User : hey Covid info bot !
<3> basicBot : Hi 
<4> User : tell me about covid 
<4> basicBot : If you have COVID-19, tell your close contacts pdf icon[93 KB, 2 Pages] you have COVID-19 so that they can quarantine at home and get tested. By letting your close contacts know they may have been exposed to COVID-19, you are helping to protect them and others within your community.
You can call, text, or email your contacts. If you would like to stay anonymous, there is also an online tool that allows you to tell your contacts by sending out emails or text notifications anonymously (www.tellyourcontacts.orgexternal icon).
<5> User : @@


In [7]:
intentExamples = {'Greetings': ['Hello !',
  'Hi , How are you ?',
  'Hey !! ',
  'Namaste ',
  'Good day',
  'Good evening',
  'Good morning',
  'Good to see you',
  'Greetings',
  'Have you been well?',
  'Hello Agent',
  'Hello',
  'Hey how are you doing',
  'Hey there all',
  'Hey there',
  'Hey twin',
  'Hey you',
  'Hi advisor',
  'Hi there',
  'How are things going?',
  'How are you today?',
  'How have you been?',
  'How is it going?',
  'How r u?',
  'Looking good eve',
  "What's new?",
  "What's up?",
  'You there',
  'Namaste',
  'satsriyakaal',
  'helo',
  'hiiiiiii',
  ' sup ',
  ' wassup bruh ',
  ' sup bro ',
  ' ssssup mate whats up',
  'hi',
  'hey'],
 'BotName': ['Who is this?',
  'Who are you',
  'Who are you ?',
  'What is your Name ?',
  'What is your name ',
  'what should i call you ?',
  'What should i call you',
  'How to address you ?',
  'how  to address you by name',
  'what is your name ',
  'call you',
  'your name',
  'call you ?',
  'your name ?',
  'whom are you',
  'whom are you referred as'],
 'Relatives': ['Do you have any relatives ',
  'Do you have a sibling ',
  'Any brother or sister',
  'Do you have a father',
  'Do you happen to have a grandparent ?',
  'Any grandparents or parents ',
  'do you have Any wife ?',
  'Are you married ?'],
 'Age': ['What is your age',
  'your age ?',
  'how old are you ?when were you born',
  'your year of birth',
  'year of birth',
  'when were you born again',
  'born ?',
  'born again ?',
  'how long have you lived ?'],
 'BirthPlace': ['where were you born ?',
  'where are you from',
  'where in the world are you from ?',
  'where do you belong to',
  'you are from ',
  'are you from ',
  'you belong to ',
  'place of birth',
  'and you are from ?',
  'and your birthplace is from ?and you are from ?',
  'birthplace',
  'place of birth',
  'location of birth',
  'place of residence',
  'location',
  'your address',
  'where do you reside ?',
  'place of residence ?'],
 'Abilities': ['What can you do ?',
  'What are your abilities ',
  'your abilities',
  'your capabilities',
  'what are your capabilities',
  'ability',
  'your ability',
  'capability',
  'your capability',
  'your abilities',
  'your powers',
  'what are your superpowers',
  'superpowers',
  'you are capable of ',
  'what are you capable of',
  'what can you do ?',
  'what can you do ?',
  'Can you jump ?',
  'could you do this ?',
  'could you please do this ?'],
 'Bye': ['good bye',
  'bye bye',
  'buh bye',
  'see you later',
  'byee',
  'se ya later aligator',
  'I gotta go',
  'I have someplace else i need to be',
  'I Need to be someplace else',
  'We can talk later',
  'Will chat later',
  'We can chat later',
  'Will be chatting later',
  'goodbye'],
 'Irrelevant': ['the weather is fine today',
  'the sun rises in the east',
  'the quick brown fox jumps over the red carpet',
  'the sun rises in the east',
  'What is love , baby dont hurt me ',
  'this is a new dawn a new day'],
 'WhatisCOVID19': ['What is COVID-19 ?',
  'What is COVID-19 ?',
  'What is COVID-19 ?',
  'What is COVID-19 ?',
  'What is COVID-19 ?',
  'What is COVID-19 ?',
  'What is COVID-19 ?',
  'What is COVID-19 ?',
  'What is COVID-19 ?',
  'What is COVID-19 ?',
   "can you tell me about covid 19 ?", #                
   #'people experience',
  #'mild symptoms',
  #'experience post',
  #'Older people',
  #'ill from COVID',
  'problems people',
  'Vaccines against COVID',
  'medical conditions',
  #'health problems people',
  #'underlying medical conditions',
  'i want information on COVID'] #
                  ,
 'Howdoesthevirusspread': ['How does the virus spread ?',
  'How does the virus spread ?',
  'How does the virus spread ?',
  'How does the virus spread ?',
  'How does the virus spread ?',
  'How does the virus spread ?',
  'How does the virus spread ?',
  'How does the virus spread ?',
  'How does the virus spread ?',
  'How does the virus spread ?',
  'breathes out droplets',
  'small particles',
  'main ways Breathing',
  'Breathing in air',
  'virus land',
  'information about how COVID',
  'exhaling small droplets',
  'Touching eyes nose',
  'surfaces they touch',
  'virus spread',
  'spread',
  'virus'],
 'Whatiscommunityspread': ['What is community spread ?',
  'What is community spread ?',
  'What is community spread ?',
  'What is community spread ?',
  'What is community spread ?',
  'What is community spread ?',
  'What is community spread ?',
  'What is community spread ?',
  'What is community spread ?',
  'What is community spread ?',
   "how does virus spread in the community ?", #  
    "tell me how covid spreads in the community ?",                       
  'does  community define how covid spreads ?',#
  #'spread differently',
  #'local conditions',
  #'local health department',
  #'department determines',
  #'virus in an area',
  #'differently based',
  #'local health',
  'information on community spread',
  'does covid spread differently based on local conditions ?', #
  'health department s website',
  'department s website',
  'does covid spread differently based on local conditions ?', #
  'information on community area spread', #
  #'based on local',
  #'visit your local health',
  #'area please visit',
  #'visit your local',
  'spread in your area',
  'community spread',
  #'spread',
  #'community'
                          ],
 'HowcanIprotectmyself': ['How can I protect myself ?',
  'How can I protect myself ?',
  'How can I protect myself ?',
  'How can I protect myself ?',
  'How can I protect myself ?',
  'How can I protect myself ?',
  'How can I protect myself ?',
  'How can I protect myself ?',
  'How can I protect myself ?',
  'How can I protect myself ?',
  'what do i do to protect myself from covid ?',#
   "how to protect oneself ?",#
   "what measures to take for protection of oneself ?"#
    "how to save self from covid ?"#                      
                         ],
 'ShouldIusesoapandwaterorhandsanitizertoprotectagainstCOVID19': ['Should I use soap and water or hand sanitizer to protect against COVID-19 ?',
  'Should I use soap and water or hand sanitizer to protect against COVID-19 ?',
  'Should I use soap and water or hand sanitizer to protect against COVID-19 ?',
  'Should I use soap and water or hand sanitizer to protect against COVID-19 ?',
  'Should I use soap and water or hand sanitizer to protect against COVID-19 ?',
  'Should I use soap and water or hand sanitizer to protect against COVID-19 ?',
  'Should I use soap and water or hand sanitizer to protect against COVID-19 ?',
  'Should I use soap and water or hand sanitizer to protect against COVID-19 ?',
  'Should I use soap and water or hand sanitizer to protect against COVID-19 ?',
  'Should I use soap and water or hand sanitizer to protect against COVID-19 ?',
  'based hand',
  'hand sanitizer',
  'Wash your hands',
  'alcohol based',
  'water are not readily',
  'bathroom and before eating',
  'hands often with soap',
  #'protect against COVID', as may confuse with HowcanIprotectmyself
  'water or hand sanitizer',
  'hand sanitizer to protect',
   #'COVID',
  'soap and water',
  'water or hand',
  'hand sanitizer',
  'sanitizer to protect',
  'soap',
  'water',
  'hand',
  'sanitizer',
  'protect'],
 'WhatshouldIdoifIgetsickorsomeoneinmyhousegetssick': ['What should I do if I get sick or someone in my house gets sick ?',
  'What should I do if I get sick or someone in my house gets sick ?',
  'What should I do if I get sick or someone in my house gets sick ?',
  'What should I do if I get sick or someone in my house gets sick ?',
  'What should I do if I get sick or someone in my house gets sick ?',
  'What should I do if I get sick or someone in my house gets sick ?',
  'What should I do if I get sick or someone in my house gets sick ?',
  'What should I do if I get sick or someone in my house gets sick ?',
  'What should I do if I get sick or someone in my house gets sick ?',
  'What should I do if I get sick or someone in my house gets sick ?',
  'develop new symptoms',
  'disease and show',
  'People who have tested',
  'vaccinated against the disease',
  'show no symptoms',
  'tested again as long',
  'identified for their symptoms',
  'quarantine or get tested',
  'house gets sick',#
   "what do i do if i get covid ?",#                                              
  'i feel sickness is spreading in my family',#
  'house'],
 'WhataretherecommendationsforsomeonewhohassymptomsofCOVID19': ['What are the recommendations for someone who has symptoms of COVID-19 ?',
  'What are the recommendations for someone who has symptoms of COVID-19 ?',
  'What are the recommendations for someone who has symptoms of COVID-19 ?',
  'What are the recommendations for someone who has symptoms of COVID-19 ?',
  'What are the recommendations for someone who has symptoms of COVID-19 ?',
  'What are the recommendations for someone who has symptoms of COVID-19 ?',
  'What are the recommendations for someone who has symptoms of COVID-19 ?',
  'What are the recommendations for someone who has symptoms of COVID-19 ?',
  'What are the recommendations for someone who has symptoms of COVID-19 ?',
  'What are the recommendations for someone who has symptoms of COVID-19 ?',
  'Wear a mask',
  'Cover your coughs',
  'coughs and sneezes',
  'nose and mouth',
  'Wash your hands',
  'Clean high',
  'surfaces every day',
  'steps below to care',
  'Avoid sharing',
  'household items',
   "what do you recommend if i have covid ?",#
   "I feel i have covid . what what should i do ?" ,#
    "I feel my brother is sick with covid ? what to do in this case ?",    #                                                              
                                                                
  #'symptoms of COVID',
  #'COVID',
  'recommendations',
  #'symptoms'
   ],
 'WhatistheriskofmychildbecomingsickwithCOVID19': ['What is the risk of my child becoming sick with COVID-19 ?',
  'What is the risk of my child becoming sick with COVID-19 ?',
  'What is the risk of my child becoming sick with COVID-19 ?',
  'What is the risk of my child becoming sick with COVID-19 ?',
  'What is the risk of my child becoming sick with COVID-19 ?',
  'What is the risk of my child becoming sick with COVID-19 ?',
  'What is the risk of my child becoming sick with COVID-19 ?',
  'What is the risk of my child becoming sick with COVID-19 ?',
  'What is the risk of my child becoming sick with COVID-19 ?',
  'What is the risk of my child becoming sick with COVID-19 ?',
  'multisystem inflammatory syndrome MIS',
  'Babies younger',
  'symptoms at all asymptomatic',
  'underlying medical conditions',
  'underlying medical',
  'medical conditions',
  'called multisystem',
  'called multisystem inflammatory',
  'developed a rare',
  'multisystem inflammatory',
  'children have developed',
  #'sick with COVID',
  #'COVID',
  'risk of my child',
  'child becoming sick',
  #'risk',
  'child',
  #'sick'
  ],
 'WhatismultisysteminflammatorysyndromeinchildrenMISC': ['What is multisystem inflammatory syndrome in children (MIS-C) ?',
  'What is multisystem inflammatory syndrome in children (MIS-C) ?',
  'What is multisystem inflammatory syndrome in children (MIS-C) ?',
  'What is multisystem inflammatory syndrome in children (MIS-C) ?',
  'What is multisystem inflammatory syndrome in children (MIS-C) ?',
  'What is multisystem inflammatory syndrome in children (MIS-C) ?',
  'What is multisystem inflammatory syndrome in children (MIS-C) ?',
  'What is multisystem inflammatory syndrome in children (MIS-C) ?',
  'What is multisystem inflammatory syndrome in children (MIS-C) ?',
  'What is multisystem inflammatory syndrome in children (MIS-C) ?',
  'syndrome in children MIS',
  'multisystem inflammatory syndrome',
  'inflammatory syndrome in children',
  'children MIS',
  'multisystem inflammatory',
  'inflammatory syndrome',
  'MIS',
  'multisystem',
  'inflammatory',
  'syndrome',
  'children'],
 'WhatarethesymptomsandcomplicationsthatCOVID19cancause': ['What are the symptoms and complications that COVID-19 can cause ?',
  'What are the symptoms and complications that COVID-19 can cause ?',
  'What are the symptoms and complications that COVID-19 can cause ?',
  'What are the symptoms and complications that COVID-19 can cause ?',
  'What are the symptoms and complications that COVID-19 can cause ?',
  'What are the symptoms and complications that COVID-19 can cause ?',
  'What are the symptoms and complications that COVID-19 can cause ?',
  'What are the symptoms and complications that COVID-19 can cause ?',
  'What are the symptoms and complications that COVID-19 can cause ?',
  'What are the symptoms and complications that COVID-19 can cause ?',
  'range of symptoms',
  'symptoms from mild',
  'symptoms to severe',
  'exposure to the virus',
  'fever cough',
  'cough or other symptoms',
  'complications that COVID',
  #'COVID',
  'symptoms and complications',
  'symptoms',
  'complications'],
 'WhenshouldIseekemergencycareifIhaveCOVID19': ['When should I seek emergency care if I have COVID-19 ?',
  'When should I seek emergency care if I have COVID-19 ?',
  'When should I seek emergency care if I have COVID-19 ?',
  'When should I seek emergency care if I have COVID-19 ?',
  'When should I seek emergency care if I have COVID-19 ?',
  'When should I seek emergency care if I have COVID-19 ?',
  'When should I seek emergency care if I have COVID-19 ?',
  'When should I seek emergency care if I have COVID-19 ?',
  'When should I seek emergency care if I have COVID-19 ?',
  'When should I seek emergency care if I have COVID-19 ?',
  'seek emergency care',
  'seek emergency',
  'emergency care',
  #'COVID',
  'seek',
  'emergency',
  'care'],
 'Isathomespecimencollectionortestingavailable': ['Is at-home specimen collection or testing available ?',
  'Is at-home specimen collection or testing available ?',
  'Is at-home specimen collection or testing available ?',
  'Is at-home specimen collection or testing available ?',
  'Is at-home specimen collection or testing available ?',
  'Is at-home specimen collection or testing available ?',
  'Is at-home specimen collection or testing available ?',
  'Is at-home specimen collection or testing available ?',
  'Is at-home specimen collection or testing available ?',
  'Is at-home specimen collection or testing available ?',
  'collect a specimen',
  'test at home',
  'local healthcare facility',
  'healthcare facility',
  'collection kit',
  'healthcare provider',
  'local healthcare',
  'signs and symptoms',
  'facility or preform',
  'preform the test',
  'information see At Home',
  'home and either send',
  'testing at a local',
  'home specimen collection',
  'specimen collection or testing',
  'home specimen',
  'specimen collection',
  'collection or testing',
  'home',
  'specimen',
  'collection',
  'testing'],
 'ShouldIbetestedforacurrentinfection': ['Should I be tested for a current infection ?',
  'Should I be tested for a current infection ?',
  'Should I be tested for a current infection ?',
  'Should I be tested for a current infection ?',
  'Should I be tested for a current infection ?',
  'Should I be tested for a current infection ?',
  'Should I be tested for a current infection ?',
  'Should I be tested for a current infection ?',
  'Should I be tested for a current infection ?',
  'Should I be tested for a current infection ?',
  'screening for COVID',
  'Test for Current',
  'check for infection Fully',
  'Testing for COVID',
  'Test for Past',
  'symptoms of COVID',
  'test result',
  'suspected or confirmed',
  'current infection',
  'tested for a current',
  'infection',
  'tested',
  'current'],
 'CansomeonetestnegativeandlatertestpositiveonaviraltestforCOVID19': ['Can someone test negative and later test positive on a viral test for COVID-19 ?',
  'Can someone test negative and later test positive on a viral test for COVID-19 ?',
  'Can someone test negative and later test positive on a viral test for COVID-19 ?',
  'Can someone test negative and later test positive on a viral test for COVID-19 ?',
  'Can someone test negative and later test positive on a viral test for COVID-19 ?',
  'Can someone test negative and later test positive on a viral test for COVID-19 ?',
  'Can someone test negative and later test positive on a viral test for COVID-19 ?',
  'Can someone test negative and later test positive on a viral test for COVID-19 ?',
  'Can someone test negative and later test positive on a viral test for COVID-19 ?',
  'Can someone test negative and later test positive on a viral test for COVID-19 ?',
  'sample was collected',
  'collected early',
  'test positive',
  'early in your infection',
  'infection and test positive',
  'steps to protect',
  'infection and test',
  'Infection for more information',
  'negative if the sample',
  'test and get infected',
  'viral test for COVID',
  'positive on a viral',
  'test negative',
  'test positive',
  'viral test',
  'test for COVID',
  #'COVID',
  'test',
  'negative and later test',
  'negative',
  'positive',
  'viral'],
 'Whatiscontacttracing': ['What is contact tracing ?',
  'What is contact tracing ?',
  'What is contact tracing ?',
  'What is contact tracing ?',
  'What is contact tracing ?',
  'What is contact tracing ?',
  'What is contact tracing ?',
  'What is contact tracing ?',
  'What is contact tracing ?',
  'What is contact tracing ?',
  'contact tracing',
  'tracing',
  'contact'],
 'Whatwillhappenwithmypersonalinformationduringcontacttracing': ['What will happen with my personal information during contact tracing ?',
  'What will happen with my personal information during contact tracing ?',
  'What will happen with my personal information during contact tracing ?',
  'What will happen with my personal information during contact tracing ?',
  'What will happen with my personal information during contact tracing ?',
  'What will happen with my personal information during contact tracing ?',
  'What will happen with my personal information during contact tracing ?',
  'What will happen with my personal information during contact tracing ?',
  'What will happen with my personal information during contact tracing ?',
  'What will happen with my personal information during contact tracing ?',
  'diagnosed with COVID',
  'personal and medical',
  'health care',
  'exposed to COVID',
  'medical information',
  'health information',
  'local health department',
  'close contact',
  'protecting health information',
  'protecting health',
  'local health',
  'notify people',
  'collecting and protecting health',
  'method for collecting',
  'information during contact tracing',
  'personal information during contact',
  'contact tracing',
  'happen with my personal',
  'personal information',
  'information during contact',
  'tracing',
  'happen',
  'personal',
  'information',
  'contact'],
 'WhoisconsideredaclosecontactofsomeonewithCOVID19': ['Who is considered a close contact of someone with COVID-19 ?',
  'Who is considered a close contact of someone with COVID-19 ?',
  'Who is considered a close contact of someone with COVID-19 ?',
  'Who is considered a close contact of someone with COVID-19 ?',
  'Who is considered a close contact of someone with COVID-19 ?',
  'Who is considered a close contact of someone with COVID-19 ?',
  'Who is considered a close contact of someone with COVID-19 ?',
  'Who is considered a close contact of someone with COVID-19 ?',
  'Who is considered a close contact of someone with COVID-19 ?',
  'Who is considered a close contact of someone with COVID-19 ?',
  'feet of an infected',
  'considered a close contact',
  #'COVID',
  'considered a close',
  'close contact',
  'considered',
  'close',
  'contact'],
 'IhaveCOVID19HowdoItellthepeopleIwasaround': ['I have COVID-19. How do I tell the people I was around ?',
  'I have COVID-19. How do I tell the people I was around ?',
  'I have COVID-19. How do I tell the people I was around ?',
  'I have COVID-19. How do I tell the people I was around ?',
  'I have COVID-19. How do I tell the people I was around ?',
  'I have COVID-19. How do I tell the people I was around ?',
  'I have COVID-19. How do I tell the people I was around ?',
  'I have COVID-19. How do I tell the people I was around ?',
  'I have COVID-19. How do I tell the people I was around ?',
  'I have COVID-19. How do I tell the people I was around ?',
  'text notifications anonymously',
  'call text',
  'text notifications',
  'stay anonymous',
  'online tool',
  'email your contacts',
  'call text or email',
  'contacts by sending',
  'emails or text notifications',
  'sending out emails',
  'text or email',
  'emails or text',
  #'COVID',
  'people'],
 'Doesmaskusehelpdetermineifsomeoneisconsideredaclosecontact': ['Does mask use help determine if someone is considered a close contact ?',
  'Does mask use help determine if someone is considered a close contact ?',
  'Does mask use help determine if someone is considered a close contact ?',
  'Does mask use help determine if someone is considered a close contact ?',
  'Does mask use help determine if someone is considered a close contact ?',
  'Does mask use help determine if someone is considered a close contact ?',
  'Does mask use help determine if someone is considered a close contact ?',
  'Does mask use help determine if someone is considered a close contact ?',
  'Does mask use help determine if someone is considered a close contact ?',
  'Does mask use help determine if someone is considered a close contact ?',
  'considered a close contact',
  'close contact',
  'mask use help determine',
  'considered a close',
  'contact',
  'mask',
  'determine',
  'considered',
  'close'],
 'IfIamaclosecontactwillIbetestedforCOVID19': ['If I am a close contact, will I be tested for COVID-19 ?',
  'If I am a close contact, will I be tested for COVID-19 ?',
  'If I am a close contact, will I be tested for COVID-19 ?',
  'If I am a close contact, will I be tested for COVID-19 ?',
  'If I am a close contact, will I be tested for COVID-19 ?',
  'If I am a close contact, will I be tested for COVID-19 ?',
  'If I am a close contact, will I be tested for COVID-19 ?',
  'If I am a close contact, will I be tested for COVID-19 ?',
  'If I am a close contact, will I be tested for COVID-19 ?',
  'If I am a close contact, will I be tested for COVID-19 ?',
  'information see COVID',
  'health department',
  'provide resources',
  'resources for testing',
  'symptoms worsen',
  'medical care',
  'monitor your symptoms',
  'seek medical',
  'seek medical care',
  'testing in your area',
  'worsen or become severe',
  'severe you should seek',
  'Watch for or monitor',
  'tested for COVID',
  'close contact',
  #'COVID',
  'close',
  'contact',
  'tested'],
 'CanIgetCOVID19frommypetsorotheranimals': ['Can I get COVID-19 from my pets or other animals ?',
  'Can I get COVID-19 from my pets or other animals ?',
  'Can I get COVID-19 from my pets or other animals ?',
  'Can I get COVID-19 from my pets or other animals ?',
  'Can I get COVID-19 from my pets or other animals ?',
  'Can I get COVID-19 from my pets or other animals ?',
  'Can I get COVID-19 from my pets or other animals ?',
  'Can I get COVID-19 from my pets or other animals ?',
  'Can I get COVID-19 from my pets or other animals ?',
  'Can I get COVID-19 from my pets or other animals ?',
  'information about pets',
  'Pets for more information',
  'pets or other animals',
  #'COVID',
  'animals',
  'pets'],
 'CananimalscarrythevirusthatcausesCOVID19ontheirskinorfur': ['Can animals carry the virus that causes COVID-19 on their skin or fur ?',
  'Can animals carry the virus that causes COVID-19 on their skin or fur ?',
  'Can animals carry the virus that causes COVID-19 on their skin or fur ?',
  'Can animals carry the virus that causes COVID-19 on their skin or fur ?',
  'Can animals carry the virus that causes COVID-19 on their skin or fur ?',
  'Can animals carry the virus that causes COVID-19 on their skin or fur ?',
  'Can animals carry the virus that causes COVID-19 on their skin or fur ?',
  'Can animals carry the virus that causes COVID-19 on their skin or fur ?',
  'Can animals carry the virus that causes COVID-19 on their skin or fur ?',
  'Can animals carry the virus that causes COVID-19 on their skin or fur ?',
  'virus that causes COVID',
  'animals carry the virus',
  'skin or fur',
  #'COVID',
  'animals carry',
  'carry the virus',
  'fur',
  'animals',
  'carry',
  'virus',
  'skin'],
 'CanIusehandsanitizeronpets': ['Can I use hand sanitizer on pets ?',
  'Can I use hand sanitizer on pets ?',
  'Can I use hand sanitizer on pets ?',
  'Can I use hand sanitizer on pets ?',
  'Can I use hand sanitizer on pets ?',
  'Can I use hand sanitizer on pets ?',
  'Can I use hand sanitizer on pets ?',
  'Can I use hand sanitizer on pets ?',
  'Can I use hand sanitizer on pets ?',
  'Can I use hand sanitizer on pets ?',
  "can i sanitize my pets ?",
   "should i be using hand sanitizer on my pets ?",                             
  'hand sanitizer on pets',
  'sanitizer on pets',
  'hand sanitizer',
  'pets',
  'hand',
  'sanitizer'],
 'WhatshouldIdoifmypetgetssickandIthinkitsCOVID19': ['What should I do if my pet gets sick and I think it’s COVID-19 ?',
  'What should I do if my pet gets sick and I think it’s COVID-19 ?',
  'What should I do if my pet gets sick and I think it’s COVID-19 ?',
  'What should I do if my pet gets sick and I think it’s COVID-19 ?',
  'What should I do if my pet gets sick and I think it’s COVID-19 ?',
  'What should I do if my pet gets sick and I think it’s COVID-19 ?',
  'What should I do if my pet gets sick and I think it’s COVID-19 ?',
  'What should I do if my pet gets sick and I think it’s COVID-19 ?',
  'What should I do if my pet gets sick and I think it’s COVID-19 ?',
  'What should I do if my pet gets sick and I think it’s COVID-19 ?',
  'contact with a person',
  'animals for COVID',
  'sick pets',
  'health concerns',
  'sick from the virus',
  'call your veterinarian',
  'sick after contact',
  'veterinary clinic',
  'evaluate your pet',
  'treatment and care',
  'offer telemedicine',
  'telemedicine consultations',
  'consultations or other plans',
  'Routine testing',
  'pet gets sick',
  #'COVID',
  'pet',
  #'sick'
  ],
 'CanwildanimalsspreadthevirusthatcausesCOVID19topeople': ['Can wild animals spread the virus that causes COVID-19 to people ?',
  'Can wild animals spread the virus that causes COVID-19 to people ?',
  'Can wild animals spread the virus that causes COVID-19 to people ?',
  'Can wild animals spread the virus that causes COVID-19 to people ?',
  'Can wild animals spread the virus that causes COVID-19 to people ?',
  'Can wild animals spread the virus that causes COVID-19 to people ?',
  'Can wild animals spread the virus that causes COVID-19 to people ?',
  'Can wild animals spread the virus that causes COVID-19 to people ?',
  'Can wild animals spread the virus that causes COVID-19 to people ?',
  'Can wild animals spread the virus that causes COVID-19 to people ?',
  'virus that causes COVID',
  'wild animals spread',
  'animals spread the virus',
  #'COVID',
  'wild animals',
  'animals spread',
  'spread the virus',
  'people',
  'wild',
  'animals',
  'spread',
  'virus'],
 'CanbatsinUnitedStatesgetthevirusthatcausesCOVID19andcantheyspreaditbacktopeople': ['Can bats in United States get the virus that causes COVID-19, and can they spread it back to people ?',
  'Can bats in United States get the virus that causes COVID-19, and can they spread it back to people ?',
  'Can bats in United States get the virus that causes COVID-19, and can they spread it back to people ?',
  'Can bats in United States get the virus that causes COVID-19, and can they spread it back to people ?',
  'Can bats in United States get the virus that causes COVID-19, and can they spread it back to people ?',
  'Can bats in United States get the virus that causes COVID-19, and can they spread it back to people ?',
  'Can bats in United States get the virus that causes COVID-19, and can they spread it back to people ?',
  'Can bats in United States get the virus that causes COVID-19, and can they spread it back to people ?',
  'Can bats in United States get the virus that causes COVID-19, and can they spread it back to people ?',
  'Can bats in United States get the virus that causes COVID-19, and can they spread it back to people ?',
  'present in bats',
  'illness or death',
  'coronavirus would make',
  'bats in the United',
  'affected by COVID',
  'bats resulting',
  'declining in the United',
  'species of bats sick',
  'important part',
  'part of natural',
  'bats in United States',
  'United States',
  'virus that causes COVID',
  'bats in United',
  'States get the virus',
  'back to people',
  'spread it back',
  #'COVID',
  'United',
  'States',
  'people',
  'bats',
  'virus',
  'spread',
  'back'],
 'Whatiscommunitymitigation': ['What is community mitigation ?',
  'tell me about community mitigation ?',
  'What is community mitigation ?',
  'What is community mitigation ?',
  'What is community mitigation ?',
  'What is community mitigation ?',
  'What is community mitigation ?',
  'What is community mitigation ?',
  'What is community mitigation ?',
  'What is community mitigation ?',
   "How to mitigate community spread of the virus",                            
  #'slow the spread',
  #'slow its spread',
  'goal of community',
  'mitigation in areas',
  'information see Community',
  'community mitigation',
  'mitigation',
  'community']}

In [8]:
from simbots.Bot import Bot
from simbots.utils.builtInIntents import IntentSamples
from simbots.utils.builtInEntities import EntitySamples
import json


newB = Bot(intentExamples, entityExamples, botMessages,testCases=testCase,
              confidenceLimit=0)

outputTheme = 'basic'

newB.run(theme = outputTheme)

Type in @@ to exit bot
@i to get intents
@c to get context 
@e to get entities
@t to add a test case
@etc to evaluate a test case
@eatc to evaluate all test cases
@ti to add intent
<1> User : tell me about covid 
<1> basicBot : COVID-19 is a disease caused by a virus called SARS-CoV-2. Most people with COVID-19 have mild symptoms, but some people can become severely ill. Although most people with COVID-19 get better within weeks of illness, some people experience post-COVID conditions. Post-COVID conditions are a wide range of new, returning, or ongoing health problems people can experience more than four weeks after first being infected with the virus that causes COVID-19. Older people and those who have certain underlying medical conditions are more likely to get severely ill from COVID-19. Vaccines against COVID-19 are safe and effective.
<2> User : can my pet get covid ?
<2> basicBot : Based on the available information to date, the risk of animals spreading COVID-19 to people is c

<7> User : thanks
<7> basicBot : Im sorry could you please rephrase ?
<8> User : @eatc
Evaluating :  faqListed
{
  "caseName": "faqListed",
  "failedCases": 0,
  "result": [
    {
      "input": "What is COVID-19 ?",
      "expectedOutput": "COVID-19 is a disease caused by a virus called SARS-CoV-2. Most people with COVID-19 have mild symptoms, but some people can become severely ill. Although most people with COVID-19 get better within weeks of illness, some people experience post-COVID conditions. Post-COVID conditions\u00a0are a wide range of new, returning, or ongoing health problems people can experience\u00a0more than four weeks after first being infected with the virus that causes COVID-19. Older people and those who have certain underlying medical conditions\u00a0are more likely to get severely ill from COVID-19. Vaccines\u00a0against COVID-19 are safe and effective.",
      "actualOutput": "COVID-19 is a disease caused by a virus called SARS-CoV-2. Most people with COVID-19 ha

<8> User : i live near bat caves can i get covid ?
<8> basicBot : Based on the available information to date, the risk of animals spreading COVID-19 to people is considered to be low. See If You Have Pets for more information about pets and COVID-19.
<9> User : @ti
Intents available are : Greetings
BotName
Relatives
Age
BirthPlace
Abilities
Bye
Irrelevant
WhatisCOVID19
Howdoesthevirusspread
Whatiscommunityspread
HowcanIprotectmyself
ShouldIusesoapandwaterorhandsanitizertoprotectagainstCOVID19
WhatshouldIdoifIgetsickorsomeoneinmyhousegetssick
WhataretherecommendationsforsomeonewhohassymptomsofCOVID19
WhatistheriskofmychildbecomingsickwithCOVID19
WhatismultisysteminflammatorysyndromeinchildrenMISC
WhatarethesymptomsandcomplicationsthatCOVID19cancause
WhenshouldIseekemergencycareifIhaveCOVID19
Isathomespecimencollectionortestingavailable
ShouldIbetestedforacurrentinfection
CansomeonetestnegativeandlatertestpositiveonaviraltestforCOVID19
Whatiscontacttracing
Whatwillhappenwithmypersonalinf

Enter intent name to update : CananimalscarrythevirusthatcausesCOVID19ontheirskinorfur
Enter the dialog number of the message to add to this intent :11
sorry this intent was not found
<11> User : 10
<11> basicBot : Im sorry could you please rephrase ?
<12> User : @ti
Intents available are : Greetings
BotName
Relatives
Age
BirthPlace
Abilities
Bye
Irrelevant
WhatisCOVID19
Howdoesthevirusspread
Whatiscommunityspread
HowcanIprotectmyself
ShouldIusesoapandwaterorhandsanitizertoprotectagainstCOVID19
WhatshouldIdoifIgetsickorsomeoneinmyhousegetssick
WhataretherecommendationsforsomeonewhohassymptomsofCOVID19
WhatistheriskofmychildbecomingsickwithCOVID19
WhatismultisysteminflammatorysyndromeinchildrenMISC
WhatarethesymptomsandcomplicationsthatCOVID19cancause
WhenshouldIseekemergencycareifIhaveCOVID19
Isathomespecimencollectionortestingavailable
ShouldIbetestedforacurrentinfection
CansomeonetestnegativeandlatertestpositiveonaviraltestforCOVID19
Whatiscontacttracing
Whatwillhappenwithmypersonali

In [9]:
newB.saveBot("faqBot_11_12_2021.p")

In [ ]:
newB.